In [1]:
##Imports

import numpy as np
import matplotlib.pyplot as plt
import requests
from sklearn.linear_model import LinearRegression,BayesianRidge
from sklearn.svm import SVR
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [2]:
##Loading training and testing data
mat_file = "mat_info.txt"
M_file_r = "echos_r.txt" # real part of echos
M_file_i = "echos_i.txt" 
mat_info = np.loadtxt(mat_file, comments="#", delimiter='\t', unpack=False);
# M(t) curve for each simulation:

##Training input

M_r = np.loadtxt(M_file_r, comments="#", delimiter='\t', unpack=False);
M_i = np.loadtxt(M_file_i, comments="#", delimiter='\t', unpack=False);

M = np.hstack((M_r, M_i)) 
##Testing Input
s_r = np.loadtxt('submit_echos_r.txt',comments="#", delimiter='\t', unpack=False)
s_i = np.loadtxt('submit_echos_i.txt',comments="#", delimiter='\t', unpack=False)
S = np.hstack((s_r, s_i)) 
print("Done with numpy loads")

Done with numpy loads


In [3]:
def submission(M,S,mat_info):
    #create training set
    predictions = np.zeros((S.shape[0], mat_info.shape[1]))
    
    for i in range(mat_info.shape[1]):
        if i==0:
            print('Processing variable 1 : Alpha')
            var1model= LinearRegression(normalize=True)
            feat_selection_var1 = SelectKBest(score_func=f_regression, k=300)
            y_train = mat_info[:,i]
            X_train = feat_selection_var1.fit_transform(M,y_train)
            print('Training....')
            var1model.fit(X_train,y_train)
            X_test = feat_selection_var1.transform(S)
            print('Testing....')
            var1_predictions = var1model.predict(X_test)
            predictions[:,i] = var1_predictions
            print(var1_predictions.shape)
        elif i==1:
            print('Processing variable 2: xi')
            var2model= BayesianRidge()
            feat_selection_var2 = SelectKBest(score_func=f_regression, k=100)
            y_train = mat_info[:,i]
            X_train = feat_selection_var2.fit_transform(M,y_train)
            print('Training....')
            var2model.fit(X_train,y_train)
            X_test = feat_selection_var2.transform(S)
            print('Testing....')
            var2_predictions = var2model.predict(X_test)
            predictions[:,i] = var2_predictions
            print(var2_predictions.shape)
        elif i==2:
            print('Processing variable 3: p')
            var3model = SVR(C=10.0, epsilon=0.0001,gamma='scale',kernel='poly')
            X_train = M
            y_train = mat_info[:,i]
            print('Training....')
            var3model.fit(X_train,y_train)
            X_test = S
            print('Testing....')
            var3_predictions = var3model.predict(X_test)
            predictions[:,i] = var3_predictions
            print(var3_predictions.shape)
        else:
            print('Processing variable 4: d')
            var4model= BayesianRidge()
            feat_selection_var4 = SelectKBest(score_func=f_regression, k=100)
            y_train = mat_info[:,i]
            X_train = feat_selection_var4.fit_transform(M,y_train)
            var4model.fit(X_train,y_train)
            print('Training....')
            X_test = feat_selection_var4.transform(S)
            print('Testing....')
            var4_predictions = var4model.predict(X_test)
            predictions[:,i] = var4_predictions
            print(var4_predictions.shape)
            print('Done....')
    return predictions

In [4]:
predictions = submission(M,S,mat_info) ##list of arrays. where each array contains the predictions of one variable

Processing variable 1 : Alpha
Training....
Testing....
(500,)
Processing variable 2: xi
Training....
Testing....
(500,)
Processing variable 3: p
Training....
Testing....
(500,)
Processing variable 4: d
Training....
Testing....
(500,)
Done....


In [9]:
np.savetxt('submitted_mat_info.txt', predictions, delimiter = '\t')

In [10]:
print('Challenge completed')
print('Team Gators4SCI')

Challenge completed
Team Gators4SCI
